In [1]:
import numpy as np
import pandas as pd
import os
from difflib import SequenceMatcher
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from itertools import chain 
import pickle

In [2]:
#-----------------------------DATENVERARBEITUNG---------------------------------------------
"""
Input: team name
Collect opponent
Output: team names of match day
"""
def get_mannschaftenTraining():
    data=pd.read_pickle("Daten161718.pkl")
    mannschaften= data[1]
    return mannschaften

"""
Input: int (number)
Colllect Data and seperate in test and training data
Output: training data, test data, list of teams
"""
def getdata(nummer):
    data=pd.read_pickle("Daten2020.pkl")
    matrix= data[0]
    mannschaften= data[2]
    schleifennummer= nummer-1
    training=[]
    test=[]
    
    #Alle Spiele vom 1. Spieltag bis zum x. Spieltag holen
    #in Trainings und Testdaten zuordnen
    for mannschaft in range(0,18):
        for spieltag in range(0,schleifennummer):
            training.append(matrix[mannschaft][spieltag])
        test.append(matrix[mannschaft][nummer-1])
    
    #Spiele der Mannschaften zuordnen, ersten x Spiele gehören Team 1 usw.
    training_dic={}
    for i in range(0,18):
        training_dic[i]= training[(nummer-1)*i:(nummer-1)*(i+1)]
    
    return training_dic,test,mannschaften

"""
Input: team name
Collect opponent
Output: team names of match day
"""
def get_mannschaften(name):
    data=pd.read_pickle(name)
    mannschaften= data[2]
    return mannschaften

"""
Auxiliary method to determine the number of matches and the time size.
Input: Desired match day, which should be predicted
Wanted to consider 5 games out of 5 games.
if this is not possible, less games will be considered for one game
worst-case: no 5 games can be considered and therefore no 5 games, but 2 and 1
output: time_size, number
"""

def size_of_time(spieltag):
    
    if spieltag >=12:
        time_size= 5
        anzahl= spieltag-6
    else:
        #spieltag=spieltag-1
        differenz1= spieltag-6
        differenz2= spieltag-5
        if differenz1 >=3: #Spieltag 9,10,11
            time_size= differenz1
            anzahl=5
        elif differenz2 ==3: #Spieltag 8
            time_size= differenz2
            anzahl=4
        elif spieltag-4==3:# Spieltag 7
            time_size= 3
            anzahl= 3
        elif spieltag-3==3: # Spieltag 6
            time_size= 2
            anzahl= 3
        elif spieltag-2==3: #Spieltag 5
            time_size= 3
            anzahl= 3
        elif spieltag-1==3: # Spieltag 4
            time_size= 2
            anzahl= 2
        elif spieltag >2: #usw.
            time_size=1
            anzahl=1
        else:
            time_size=0
            anzahl=0
    return time_size,anzahl

"""
input; data, team
filter the statistics of the respective team
output: List of statistics from the games
"""
def get_statistik(data, team):
    liste=[]
    
    for i in data:
        teams= i[-2:]
        werte= i[:-2]
        if teams[0]==team:
            liste.append(werte[19:32])
        else:
            liste.append(werte[51:64])
    return liste

"""
input: team, list of teams
Similarity search of the teams
ouput: right team name
"""
def similar(mannschaft, liste): 
    wert=0
    currentteam=""
    
    for team in liste:
        similarity= SequenceMatcher(None, mannschaft, team).ratio()
        #print(team, similarity)
        if wert < similarity:
            wert= similarity
            currentteam= team
            
    return currentteam

"""
input: data, lis of team, team, matchday number, promoted team, relegated team
Reduction of the data
If there is a promoted team at the queried game
you can't look at the previous season, because there is only one team there
So --> remove data of the previous season
If there is a relegated team at the queried game
--> Error message
This is done for 3 seasons(so 2 promoted/relegated)
output: Data, new game day(instead of 40, 40-34=6).
"""
def data_auf_absteiger(data,mannschaftsliste,mannschaft,spieltag,absteiger,aufsteiger):

    #Indices der spielenenden Mannschaften herausfinden
    current_team1=similar(mannschaft[0],mannschaftsliste)
    index1= mannschaftsliste.index(current_team1)
    
    current_team2=similar(mannschaft[1],mannschaftsliste)
    index2= mannschaftsliste.index(current_team2)
    
    #Zusammenführung der Auf- bzw. Absteiger in einer Liste für Abfrage
    auf= list(chain.from_iterable(list(aufsteiger.values())))
    ab= list(chain.from_iterable(list(absteiger.values())))
    
    if spieltag >34 and (index1 in auf or index2 in auf or index1 in ab or index2 in ab):
        
        #Saison 2
        if spieltag <69:
            #Auf-/Absteiger von Saison 3 entfernen
            del absteiger[1]
            del aufsteiger[2]
            #Schauen ob Mannschaft Auf-/Absteiger ist, zweite Saison
            
            # Wenn sich um Absteiger handelt, ist dieser nicht in aktueller Saison --> Fehler
            for key,values in absteiger.items():
                if index1 in values or index2 in values:
                    return "KEY ERROR",""
            #Wenn sich um Aufsteiger handelt 
            #--> Saison 1 aus Daten nehmen und Spieltag aktualisiern
            for key,values in aufsteiger.items():
                if index1 in values or index2 in values:
                    for key, values in data.items():
                        data.update({key:values[34:]})
                    return data, spieltag-34
                else:
                    return data, spieltag
        #Saison 3
        if spieltag >=69:
            #Absteiger entfernen
            del absteiger[0]
            
            # Wenn sich um Absteiger handelt, ist dieser nicht in aktueller Saison --> Fehler
            for key,values in absteiger.items():
                if index1 in values or index2 in values:
                    return "KEY ERROR",""
            #Wenn sich um Aufsteiger handelt 
            #--> Ab Saison 1 bzw. ab Saison 2 aus Daten nehmen und Spieltag aktualisiern
            for key,values in aufsteiger.items():
                if index1 in values or index2 in values:
                    if key==1: #Aufsteiger von Sasion2
                        for key, values in data.items():
                            data.update({key:values[68:]})
                        return data, spieltag-68
                    else: #Aufsteiger von Saison3
                        for key, values in data.items():
                            data.update({key:values[68:]})
                        return data, spieltag-68
    else:# Fall für Saison1 ==> keine Reduzierung notwendig
        return data, spieltag
    
"""
input: data, team, list of team
Find out the opponent of the passed team
Go through data and find team. There is also the opponentID
output: Name of opponent and location(is team home or visiting team).
"""
def getgegner(data,teamname,mannschaftsliste):
    current_team=similar(teamname,mannschaftsliste)
    key= mannschaftsliste.index(current_team)
    
    #Gegner finden
    for i in data:
        teams= i[-2:]
        if key in teams:
            current_teams= teams
            break
    #Heim oder Auswärtsmannschaft
    if list(current_teams).index(key)==0:
        ort= "home"
    else:
        ort= "away"
    
    #Name des Gegner bekommen
    gegner= current_teams[current_teams!=key][0]
    gegnername= mannschaftsliste[int(gegner)]
    return gegnername,ort




#--------------------------TIMESEQUENZEN/AUXDATA TRAINING---------------------------------



"""
input: data, list of team, team, match day
Create time sequences
In the best case the last 5 game statistics are considered per game
How many are looked at is calculated with size_of_time
Now get x statistics from the team and its opponents
Save them in a list and consider the next game day
output: List of TimeSequences, size of each TimeSequence, team key.
"""
def time_sequence(data,mannschaftsliste,mannschaft,spieltag):
    time_size, anzahl= size_of_time(spieltag)
    time_series={}
    list_of_game_series=[]
    grenze= spieltag-1
    
    #Mannschaftnummer finden
    current_team=similar(mannschaft,mannschaftsliste)
    #TimeSequenzen erstellen
    for key,values in data.items():
        if key!= mannschaftsliste.index(current_team):
            continue
        else:
            for i in range(time_size,grenze): #Spiele die betrachtet werden
                teams= values[i][-2:]
                start_gegner= i-time_size
                if teams[0]==key:
                    #"Rohe" Daten holen
                    gegner_statistik= data[teams[1]][start_gegner:i]
                    team_statistik= data[teams[0]][start_gegner:i]
                    #Filterung der Statistiken
                    team_statistik= get_statistik(team_statistik,teams[0])
                    gegner_statistik= get_statistik(gegner_statistik,teams[1])
                    #Zusammenführung
                    game_series= np.concatenate((team_statistik, gegner_statistik), axis=1)
                else:
                    #"Rohe" Daten holen
                    gegner_statistik= data[teams[0]][start_gegner:i]
                    team_statistik= data[teams[1]][start_gegner:i]              
                    #Filterung der Statistiken
                    team_statistik= get_statistik(team_statistik,teams[1])
                    gegner_statistik= get_statistik(gegner_statistik,teams[0])
                    #Zusammenführung
                    game_series= np.concatenate((gegner_statistik, team_statistik), axis=1)
                list_of_game_series.append(game_series)
                
    mannschaftskey= mannschaftsliste.index(current_team)
    return np.array(list_of_game_series), mannschaftskey, time_size

"""
input: data, key of team, matchday
Create auxiliary data, i.e. table, team, player values, betting odds.
extract output for these matches as well
se #size of time to find considered games
create list for aux_data and output
output: list of aux data and list of output values
"""
def auxilary_data(data,mannschaftskey,spieltag):
    time_size, anzahl= size_of_time(spieltag)
    aux_data={}
    out_data={}
    grenze= spieltag-1
    
    for key,values in data.items():
        list_of_game_series=[]
        list_of_outputs=[]
        if key!= mannschaftskey:
            continue
        else:
            for i in range(time_size,grenze):
                teams= values[i][-2:]
                start_gegner= i-time_size
                #Aux_Data und Output holen
                home_team= values[i][:19]
                away_team= values[i][32:51]
                odds= values[i][64:67]
                output= values[i][67:70]
                game_data= np.concatenate((home_team,away_team,odds))
                #Speichern in Listen
                list_of_outputs.append(output)
                list_of_game_series.append(game_data)
            #Separierung der Mannschaften durch dictionary
            aux_data[key]=list_of_game_series
            out_data[key]=list_of_outputs
            
    return list(aux_data.values()),list(out_data.values())

"""        
input: outputs
convert output into single values, [1,0,0]=1,[0,1,0]=0,[0,0,1]=2
output: array with single values instead of single lists
"""
def out_data(outputs):
    output=[]
    
    for i in outputs:
        if i[0]==1:
            output.append(0)
        elif i[1]==1:
            output.append(1)
        else:
            output.append(2)
    return np.array(output)




#-----------------------TIMESEQUENZEN/AUXDATA FÜR DEN TEST----------------------------------


"""
input: trainings data, test data, team, match day
Create TimeSequence for the test data
Difference one looks only at the one game
First find out home and away team
Then create the TimeSequence for both teams
loop does not run, because the game is twice in the list
output: array with time sequences
"""
def time_sequence_test(data_training,data_test,team,spieltag):

    time_size, anzahl= size_of_time(spieltag)
    liste=[]
    
    #Herausfindeen ob Team Heim oder Auswärtsmannschaft ist
    for spiele in data_test:
        mannschaften= spiele[-2:]
        if team in mannschaften:
            if mannschaften[0]==team:
                gegner= [mannschaften[1],"away"]
            else:
                gegner= [mannschaften[0],"home"]
            break
    
    #TimeSequenz erstellen
    for key,values in data_training.items():
        if key!= team:
            continue
        else:

            gegner_statistik= data_training[gegner[0]][-time_size:]
            team_statistik= data_training[team][-time_size:]
            #Filterung der Statistiken
            team_statistik= get_statistik(team_statistik,team)
            gegner_statistik= get_statistik(gegner_statistik,gegner[0])
            #Zusammenführung und Reihenfolge beachten
            if gegner[1]=="away":
                game_series= np.concatenate((team_statistik, gegner_statistik), axis=1)
            else:
                game_series= np.concatenate((gegner_statistik, team_statistik), axis=1)
            liste.append(game_series)
            return np.array(liste)

"""
input: test data, team
Fetch auxilary data for test
Simply fetch these values for a game
loop does not run because the game is in the list twice
output: List of Aux_Data and list of playing teams.
"""n
def auxilary_data_test(data_test,team):
    liste=[]
    for spiele in data_test:
        mannschaften= spiele[-2:] #Mannschaften bekommen
        if team in mannschaften: #Mannschaft finden
            #Werte holen
            home_team= spiele[:19]
            away_team= spiele[32:51]
            odds= spiele[64:67]
            output= spiele[67:70]
            game_data= np.concatenate((home_team,away_team,odds))
            liste.append(game_data)
            return liste, mannschaften, output

        
        
        
        
#---------------------------------SPEZIALFÄLLE-----------------------------------------------

"""
input: data, test data, list of team, team
get all data, so TimeSequence, Aux_data, Output for the 2.GAME DAY
is a special case because you can't build a real TimeSequence because you know only one game #from past
knowing from the past
output: All data: (TimeSequence, Aux, Output)*2 (for both teams), Key from team
"""
def getspieltag2(data,test,mannschaftsliste,mannschaft):
    current_team=similar(mannschaft,mannschaftsliste)
    liste_time_series=[]
    liste_aux=[]
    liste_time_series_a=[]
    liste_aux_a=[]
    output=[]
    output_a=[]
    
    #Mannschaften holen
    for spiele in test:
        mannschaften=spiele[-2:]
        if mannschaftsliste.index(current_team) in mannschaften:
            current_teams= mannschaften
            break
    
    #Daten für Mannschaften holen
    for key,values in data.items():
        for i in current_teams:
            if key==i and i == current_teams[0]:
                #TimeSequenz
                liste_time_series.append(values[0][19:32])
                liste_time_series.append(values[0][51:64])
                liste_aux.append(values[0][:19])
                liste_aux.append(values[0][32:51])
                liste_aux.append(values[0][64:67])
                output.append(values[0][67:70])
            if key==i and i == current_teams[1]:
                liste_time_series_a.append(values[0][19:32])
                liste_time_series_a.append(values[0][51:64])
                liste_aux_a.append(values[0][:19])
                liste_aux_a.append(values[0][32:51])
                liste_aux_a.append(values[0][64:67])
                output_a.append(values[0][67:70])
                
    liste_time_series= list(chain.from_iterable(liste_time_series))
    liste_aux= list(chain.from_iterable(liste_aux))
    liste_time_series_a= list(chain.from_iterable(liste_time_series_a))
    liste_aux_a= list(chain.from_iterable(liste_aux_a))
    
    return np.array([[liste_time_series]]), np.array([liste_aux]),np.array(output),np.array([[liste_time_series_a]]),np.array([liste_aux_a]), np.array(output_a), mannschaftsliste.index(current_team)

"""
input: team name, list of team, test data
match day 1
Here you have no TIMESEQUENCES --> create placeholder (-1)
get auxdata completelynormal and at predict again the same input as at fit
output: Team1,Team2, Prediction
"""
def spieltag1(teamname,mannschaftsliste,test):
    current_team=similar(teamname,mannschaftsliste)
    key= mannschaftsliste.index(current_team)
    aux_data_test,mannschaften, output=auxilary_data_test(test,key)
    aux_data= np.array(aux_data_test)
    time= np.array([[[-1]*26]*5]*1) #np.full

    out= output1(test,key)

    return aux_data,time,out


"""
input: test, team
Output conversion for the first match day
output: convert output
"""
def output1(test, mannschaft):
    for i in test:
        if mannschaft in i[-2:]:
            out= i[-5:-2]
            break
    if out[0]==1:
        return np.array([0])
    elif out[1]==1:
        return np.array([1])
    else: 
        return np.array([2])

In [1]:
"""
input: match day, team
This method generates the time series and additional data for one matchday and one team.
output:test data, opponent
"""
def create_data(spieltag, mannschaft):
    spieltag=spieltag
    teamname=mannschaft
    training, test, mannschaftsliste= getdata(spieltag)
    teamname_gegner,ort= getgegner(test,teamname,mannschaftsliste)    

    
    #Sonederfall
    if spieltag==1:
        aux,time,out= spieltag1(teamname,mannschaftsliste,test)
        teamname_gegner,ort= getgegner(test,teamname,mannschaftsliste)
        if out==[0]:
            out=[1,0,0]
        elif out==[1]:
            out=[0,1,0]
        else:
            out=[0,0,1]
            
        return aux,time,out,ort,teamname_gegner
        
    #Hier wird nur der Key geholt    
    elif spieltag>2:
        time2vec,key, seq_len= time_sequence(training,mannschaftsliste,teamname,spieltag)
    else: 
        time2vec, aux_data, output,time2vec_a,aux_data_a,output_a,key= getspieltag2(training,test,mannschaftsliste,teamname)

    #Daten holen
    time2vectest= time_sequence_test(training,test,key,spieltag)
    aux_data_test,mannschaften,output_test=auxilary_data_test(test,key)
    aux_data_test=np.array(aux_data_test)
    if time2vec[0].shape[0]<5:
        
        size= 5-time2vectest[0].shape[0]
        anzahl=time2vectest.shape[0]
        time2vec_neu= np.array(anzahl*[5*[[-1.0]*26]])
        time2vec_neu[0]= np.concatenate((np.array(size*[[-1]*26]),time2vectest[0]),axis=0)

    else:
        time2vec_neu= time2vectest
    
    return aux_data_test,time2vec_neu,output_test,ort,teamname_gegner

"""
input: model, dataname
Prediction is made for each team+game day.
Load model, create inputs, save results
output: pkl file
"""
def evaluate(model,dateiname):
    teams= get_mannschaften("Daten2020.pkl")
    sammlung_mannschaften= {}
    new_model = tf.keras.models.load_model(model)
    for mannschaft in teams:
        sammlung_spieltage= {}
        for spieltag in range(1,35):
            
            aux_data_test,time2vec_neu,output_test,ort,teamname_gegner= create_data(spieltag,mannschaft)
            output= new_model.predict([time2vec_neu,aux_data_test])[0]
            #print(new_model.summary())
            
            if ort=="home":
                ergebnis= [mannschaft,teamname_gegner,output,output_test]
            else:
                ergebnis= [teamname_gegner,mannschaft,output,output_test]
            
            liste=[]
            liste.append(ergebnis)
            sammlung_spieltage[spieltag]= liste
    
        sammlung_mannschaften[mannschaft]= sammlung_spieltage
    
    with open(dateiname,'wb') as f: 
        pickle.dump(sammlung_mannschaften, f)  
#evaluate("Transformer4Saisons","TransformerAttn.pkl")

In [3]:
"""
input: -
Output of predictions
"""
def Ausgabe():
    teams= get_mannschaften("Daten2020.pkl")
    data=pd.read_pickle("Transformer4Ergebnisse.pkl")
    data= data[teams[0]]

    count=1
    for k,v in data.items():
        for i in v:    
            print(count, i,"\n")
        count+=1